In [31]:
import tensorflow as tf
tf.__version__


'2.12.0'

In [32]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [33]:
subject = 'Pernambuco - Consumo de Cimento (t)'
split_index = 6 #Referente aos 7 anos de input  

In [34]:
data = pd.read_csv('../Inputs Atuais/2014_01_model_input_PE.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data

,Ano,Pernambuco - IDH Longevidade,Pernambuco - Desemprego,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pernambuco - IDH Educacao,Pernambuco - IDH,Pernambuco - Consumo de Cimento (t)
0,2014,0.795030,13.857509,9.733647e+07,5.498393e+06,8.923227,7.929548e+07,0.666934,5.676413e+08,2.779340e+09,...,19.852601,1013.159804,1654.440407,25.780576,20.818540,75.113876,1.558118,0.684539,0.713117,2983.632857
1,2015,0.809436,14.543115,9.797155e+07,5.411870e+06,8.896326,7.910750e+07,0.668176,5.179392e+08,2.561731e+09,...,18.352626,1013.187593,1731.551083,25.909409,21.066299,75.720358,1.597579,0.700173,0.723266,2632.837886
2,2016,0.809748,14.833751,9.849009e+07,5.357988e+06,8.887508,7.905537e+07,0.666374,4.524144e+08,2.516859e+09,...,16.821327,1013.081354,1774.985187,25.897163,21.315125,76.984087,1.606812,0.702189,0.723381,2308.455257
3,2017,0.809452,14.580259,9.859294e+07,5.346247e+06,8.880979,7.898921e+07,0.665408,4.213689e+08,2.497710e+09,...,16.755645,1013.025277,1768.623220,25.854367,21.296659,77.129112,1.578862,0.703297,0.723324,1813.941000
4,2018,0.808584,14.328297,9.811873e+07,5.353096e+06,8.858194,7.873031e+07,0.664900,4.166580e+08,2.797145e+09,...,17.559274,1012.929728,1693.912882,25.973800,21.376614,76.977283,1.898908,0.705051,0.723445,1645.004000
5,2019,0.807001,14.284157,9.731261e+07,5.380332e+06,8.835747,7.848506e+07,0.664527,4.462625e+08,3.484252e+09,...,18.069987,1012.775721,1612.605210,26.025630,21.470428,77.226509,2.155096,0.703783,0.722394,1690.846000
6,2020,0.804820,14.265196,9.589838e+07,5.418959e+06,8.804547,7.811752e+07,0.663121,5.489412e+08,4.114911e+09,...,17.152620,1012.632277,1589.133564,25.844735,21.719349,79.332985,1.974684,0.702491,0.720794,2018.834000
7,2021,0.802440,14.240429,9.432318e+07,5.466622e+06,8.803752,7.798536e+07,0.662768,6.638811e+08,4.421848e+09,...,14.413007,1012.278387,1603.093434,25.526054,22.167765,83.147453,1.671540,0.700081,0.719139,2022.411000
8,2022,0.802934,14.265103,9.240245e+07,5.522973e+06,8.801473,7.780811e+07,0.664201,7.010332e+08,4.533216e+09,...,12.830100,1011.919754,1634.949995,25.367655,22.441498,85.329090,1.547031,0.698179,0.719151,1940.927000


In [35]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Pernambuco - IDH Longevidade,Pernambuco - Desemprego,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Pernambuco - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pernambuco - IDH Educacao,Pernambuco - IDH
0,0.795030,13.857509,9.733647e+07,5.498393e+06,8.923227,7.929548e+07,0.666934,5.676413e+08,2.779340e+09,1.954361e+10,...,988.207737,19.852601,1013.159804,1654.440407,25.780576,20.818540,75.113876,1.558118,0.684539,0.713117
1,0.809436,14.543115,9.797155e+07,5.411870e+06,8.896326,7.910750e+07,0.668176,5.179392e+08,2.561731e+09,1.327596e+10,...,808.858465,18.352626,1013.187593,1731.551083,25.909409,21.066299,75.720358,1.597579,0.700173,0.723266
2,0.809748,14.833751,9.849009e+07,5.357988e+06,8.887508,7.905537e+07,0.666374,4.524144e+08,2.516859e+09,1.011370e+10,...,597.562419,16.821327,1013.081354,1774.985187,25.897163,21.315125,76.984087,1.606812,0.702189,0.723381
3,0.809452,14.580259,9.859294e+07,5.346247e+06,8.880979,7.898921e+07,0.665408,4.213689e+08,2.497710e+09,8.456309e+09,...,438.177864,16.755645,1013.025277,1768.623220,25.854367,21.296659,77.129112,1.578862,0.703297,0.723324
4,0.808584,14.328297,9.811873e+07,5.353096e+06,8.858194,7.873031e+07,0.664900,4.166580e+08,2.797145e+09,7.937760e+09,...,386.774755,17.559274,1012.929728,1693.912882,25.973800,21.376614,76.977283,1.898908,0.705051,0.723445
5,0.807001,14.284157,9.731261e+07,5.380332e+06,8.835747,7.848506e+07,0.664527,4.462625e+08,3.484252e+09,1.091282e+10,...,384.698153,18.069987,1012.775721,1612.605210,26.025630,21.470428,77.226509,2.155096,0.703783,0.722394
6,0.804820,14.265196,9.589838e+07,5.418959e+06,8.804547,7.811752e+07,0.663121,5.489412e+08,4.114911e+09,1.625333e+10,...,360.254964,17.152620,1012.632277,1589.133564,25.844735,21.719349,79.332985,1.974684,0.702491,0.720794


In [36]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0    2632.837886
1    2308.455257
2    1813.941000
3    1645.004000
4    1690.846000
5    2018.834000
6    2022.411000
7    1940.927000
8            NaN
Name: Pernambuco - Consumo de Cimento (t), dtype: float64

In [37]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Pernambuco - IDH Longevidade,Pernambuco - Desemprego,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Pernambuco - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pernambuco - IDH Educacao,Pernambuco - IDH
0,-2.201720,-1.800410,-1.261897,2.034928,1.544949,1.327563,0.701158,1.779974,0.019134,2.003355,...,1.698361,1.854919,0.946044,-0.874758,-1.599254,-1.854055,-1.748050,-0.787667,-2.184173,-2.226106
1,0.553439,0.456188,0.002285,0.390539,0.576107,0.617907,1.690733,0.870373,-0.621387,0.401153,...,0.912286,0.428587,1.143377,0.433000,0.032747,-0.720967,-0.996870,-0.609489,0.047745,0.472788
2,0.613175,1.412784,1.034503,-0.633497,0.258526,0.421101,0.255587,-0.328802,-0.753466,-0.407217,...,-0.013810,-1.027530,0.388960,1.169620,-0.122383,0.417004,0.568365,-0.567799,0.335507,0.503387
3,0.556635,0.578443,1.239238,-0.856644,0.023357,0.171340,-0.513672,-0.896969,-0.809829,-0.830898,...,-0.712381,-1.089987,-0.009249,1.061724,-0.664505,0.332549,0.747991,-0.694000,0.493693,0.488416
4,0.390635,-0.250862,0.295267,-0.726478,-0.797245,-0.806038,-0.918489,-0.983185,0.071542,-0.963455,...,-0.937677,-0.325814,-0.687755,-0.205325,0.848419,0.698213,0.559939,0.751096,0.744129,0.520437
5,0.087837,-0.396143,-1.309396,-0.208847,-1.605694,-1.731873,-1.215316,-0.441390,2.094006,-0.202939,...,-0.946779,0.159825,-1.781378,-1.584261,1.504977,1.127256,0.868625,1.907859,0.563100,0.241078


In [38]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0    2632.837886
1    2308.455257
2    1813.941000
3    1645.004000
4    1690.846000
5    2018.834000
Name: Pernambuco - Consumo de Cimento (t), dtype: float64

In [39]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Pernambuco - IDH Longevidade,Pernambuco - Desemprego,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Pernambuco - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pernambuco - IDH Educacao,Pernambuco - IDH
6,-0.302648,-0.418301,-2.061765,0.476989,-1.758782,-1.868052,-1.621024,1.16956,2.035192,0.985175,...,-0.906462,-0.636957,-1.780396,-1.468752,-0.697983,1.593244,1.949457,0.935438,0.346973,-0.170454


In [40]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

6    2022.411
Name: Pernambuco - Consumo de Cimento (t), dtype: float64

In [41]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=500, verbose=1)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(148, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(148, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(76, activation='tanh'),
        tf.keras.layers.Dense(36, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=5000, 
                      batch_size=len(train_input), 
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [42]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_val_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, 0, seed)

        current_val_loss = min(history.history['val_loss'])
        print(f"val_loss: {current_val_loss}")

        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [43]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1244765141, 2629909586, 2377215264, 3487728531, 381957026, 2519180607, 500374453, 1849689994, 544025979, 4073639941]


Step: 0 ___________________________________________


2023-08-20 10:28:18.563369: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-20 10:28:18.563736: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1593: early stopping
val_loss: 2460.2666015625
winner_seed: 1244765141


Step: 1 ___________________________________________
Epoch 916: early stopping
val_loss: 4753.8935546875


Step: 2 ___________________________________________
Epoch 1567: early stopping
val_loss: 3049.932861328125


Step: 3 ___________________________________________
Epoch 1509: early stopping
val_loss: 1.437247395515442
winner_seed: 3487728531


Step: 4 ___________________________________________
Epoch 922: early stopping
val_loss: 4387.1865234375


Step: 5 ___________________________________________
Epoch 1502: early stopping
val_loss: 9243.935546875


Step: 6 ___________________________________________
Epoch 1229: early stopping
val_loss: 14757.2666015625


Step: 7 ___________________________________________
Epoch 2348: early stopping
val_loss: 12.098372459411621


Step: 8 ___________________________________________
Epoch 1741: early stopping
val_loss: 4.2267584800720215


Step: 9 _________________________

In [44]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/5000
1/1 [==============================] - 0s 304ms/step - loss: 4199118.5000 - val_loss: 3920545.5000
Epoch 2/5000
1/1 [==============================] - 0s 13ms/step - loss: 4167147.7500 - val_loss: 3068985.5000
Epoch 3/5000
1/1 [==============================] - 0s 13ms/step - loss: 3935803.0000 - val_loss: 4664266.0000
Epoch 4/5000
1/1 [==============================] - 0s 12ms/step - loss: 3815910.0000 - val_loss: 4502428.0000
Epoch 5/5000
1/1 [==============================] - 0s 12ms/step - loss: 3503035.7500 - val_loss: 5048422.0000
Epoch 6/5000
1/1 [==============================] - 0s 12ms/step - loss: 3723550.2500 - val_loss: 4895171.5000
Epoch 7/5000
1/1 [==============================] - 0s 11ms/step - loss: 3411541.7500 - val_loss: 4436815.5000
Epoch 8/5000
1/1 [==============================] - 0s 12ms/step - loss: 3172920.7500 - val_loss: 4269030.0000
Epoch 9/5000
1/1 [==============================] - 0s 12ms/step - loss: 3331426.2500 - val_loss: 4070327.0000


1/1 [==============================] - 0s 12ms/step - loss: 1019350.0000 - val_loss: 1417226.2500
Epoch 75/5000
1/1 [==============================] - 0s 12ms/step - loss: 1002330.7500 - val_loss: 1401182.8750
Epoch 76/5000
1/1 [==============================] - 0s 13ms/step - loss: 1100514.7500 - val_loss: 1386789.1250
Epoch 77/5000
1/1 [==============================] - 0s 12ms/step - loss: 1039791.0000 - val_loss: 1359055.6250
Epoch 78/5000
1/1 [==============================] - 0s 12ms/step - loss: 1135113.8750 - val_loss: 1284579.2500
Epoch 79/5000
1/1 [==============================] - 0s 12ms/step - loss: 932353.5000 - val_loss: 1356007.1250
Epoch 80/5000
1/1 [==============================] - 0s 16ms/step - loss: 1087084.8750 - val_loss: 1286877.5000
Epoch 81/5000
1/1 [==============================] - 0s 17ms/step - loss: 1033010.7500 - val_loss: 1328326.6250
Epoch 82/5000
1/1 [==============================] - 0s 16ms/step - loss: 963394.1875 - val_loss: 1293562.5000
Epoch 83

Epoch 148/5000
1/1 [==============================] - 0s 11ms/step - loss: 325181.7500 - val_loss: 484881.5312
Epoch 149/5000
1/1 [==============================] - 0s 12ms/step - loss: 380057.0000 - val_loss: 508448.2188
Epoch 150/5000
1/1 [==============================] - 0s 11ms/step - loss: 303804.5938 - val_loss: 568442.7500
Epoch 151/5000
1/1 [==============================] - 0s 11ms/step - loss: 313578.1562 - val_loss: 569094.6875
Epoch 152/5000
1/1 [==============================] - 0s 11ms/step - loss: 333180.0625 - val_loss: 580642.8750
Epoch 153/5000
1/1 [==============================] - 0s 11ms/step - loss: 332584.3438 - val_loss: 568382.5625
Epoch 154/5000
1/1 [==============================] - 0s 11ms/step - loss: 386824.5938 - val_loss: 557342.8125
Epoch 155/5000
1/1 [==============================] - 0s 12ms/step - loss: 328941.6250 - val_loss: 557655.0625
Epoch 156/5000
1/1 [==============================] - 0s 12ms/step - loss: 384852.9062 - val_loss: 540275.4375
E

Epoch 222/5000
1/1 [==============================] - 0s 13ms/step - loss: 120560.7734 - val_loss: 338901.6250
Epoch 223/5000
1/1 [==============================] - 0s 11ms/step - loss: 111131.2891 - val_loss: 347974.5625
Epoch 224/5000
1/1 [==============================] - 0s 12ms/step - loss: 129814.3516 - val_loss: 333700.1562
Epoch 225/5000
1/1 [==============================] - 0s 12ms/step - loss: 127467.5234 - val_loss: 328900.0000
Epoch 226/5000
1/1 [==============================] - 0s 12ms/step - loss: 128501.8750 - val_loss: 315177.2500
Epoch 227/5000
1/1 [==============================] - 0s 11ms/step - loss: 143430.8750 - val_loss: 318906.4375
Epoch 228/5000
1/1 [==============================] - 0s 16ms/step - loss: 126911.4141 - val_loss: 322794.8750
Epoch 229/5000
1/1 [==============================] - 0s 16ms/step - loss: 158512.3438 - val_loss: 325715.1562
Epoch 230/5000
1/1 [==============================] - 0s 16ms/step - loss: 105577.0547 - val_loss: 335308.8438
E

1/1 [==============================] - 0s 14ms/step - loss: 74846.9453 - val_loss: 215836.1250
Epoch 297/5000
1/1 [==============================] - 0s 11ms/step - loss: 69177.8359 - val_loss: 210587.5469
Epoch 298/5000
1/1 [==============================] - 0s 11ms/step - loss: 68540.2422 - val_loss: 206646.4844
Epoch 299/5000
1/1 [==============================] - 0s 11ms/step - loss: 67955.6797 - val_loss: 203585.2500
Epoch 300/5000
1/1 [==============================] - 0s 12ms/step - loss: 69507.1406 - val_loss: 199011.3906
Epoch 301/5000
1/1 [==============================] - 0s 12ms/step - loss: 66730.9141 - val_loss: 199423.5156
Epoch 302/5000
1/1 [==============================] - 0s 11ms/step - loss: 43615.7773 - val_loss: 225823.1094
Epoch 303/5000
1/1 [==============================] - 0s 11ms/step - loss: 69365.7734 - val_loss: 212856.2969
Epoch 304/5000
1/1 [==============================] - 0s 11ms/step - loss: 66100.2656 - val_loss: 204989.4219
Epoch 305/5000
1/1 [=====

...

Epoch 1198/5000
1/1 [==============================] - 0s 11ms/step - loss: 2908.9531 - val_loss: 43835.9492
Epoch 1199/5000
1/1 [==============================] - 0s 11ms/step - loss: 5923.4668 - val_loss: 44737.3086
Epoch 1200/5000
1/1 [==============================] - 0s 11ms/step - loss: 9956.6504 - val_loss: 49647.0430
Epoch 1201/5000
1/1 [==============================] - 0s 11ms/step - loss: 3575.4832 - val_loss: 52811.8789
Epoch 1202/5000
1/1 [==============================] - 0s 11ms/step - loss: 13328.5088 - val_loss: 47112.7383
Epoch 1203/5000
1/1 [==============================] - 0s 11ms/step - loss: 7766.4448 - val_loss: 47180.7500
Epoch 1204/5000
1/1 [==============================] - 0s 11ms/step - loss: 10797.3633 - val_loss: 39607.7031
Epoch 1205/5000
1/1 [==============================] - 0s 11ms/step - loss: 3389.0662 - val_loss: 37994.4414
Epoch 1206/5000
1/1 [==============================] - 0s 11ms/step - loss: 6718.2104 - val_loss: 38761.4961
Epoch 1207/5000
1

Epoch 1273/5000
1/1 [==============================] - 0s 12ms/step - loss: 14174.9053 - val_loss: 36132.5703
Epoch 1274/5000
1/1 [==============================] - 0s 12ms/step - loss: 8049.6094 - val_loss: 30169.6367
Epoch 1275/5000
1/1 [==============================] - 0s 13ms/step - loss: 6815.2988 - val_loss: 26134.4395
Epoch 1276/5000
1/1 [==============================] - 0s 12ms/step - loss: 7169.1372 - val_loss: 32364.7656
Epoch 1277/5000
1/1 [==============================] - 0s 12ms/step - loss: 10645.9189 - val_loss: 31853.0547
Epoch 1278/5000
1/1 [==============================] - 0s 11ms/step - loss: 12480.9805 - val_loss: 32447.9629
Epoch 1279/5000
1/1 [==============================] - 0s 16ms/step - loss: 28251.7090 - val_loss: 27272.8770
Epoch 1280/5000
1/1 [==============================] - 0s 12ms/step - loss: 6846.5601 - val_loss: 24552.8477
Epoch 1281/5000
1/1 [==============================] - 0s 12ms/step - loss: 10880.8242 - val_loss: 25559.3516
Epoch 1282/500

1/1 [==============================] - 0s 11ms/step - loss: 20951.0801 - val_loss: 38027.4297
Epoch 1348/5000
1/1 [==============================] - 0s 11ms/step - loss: 10861.0859 - val_loss: 28535.2188
Epoch 1349/5000
1/1 [==============================] - 0s 11ms/step - loss: 11581.5947 - val_loss: 26703.9160
Epoch 1350/5000
1/1 [==============================] - 0s 12ms/step - loss: 19805.2109 - val_loss: 24396.7852
Epoch 1351/5000
1/1 [==============================] - 0s 11ms/step - loss: 10964.5938 - val_loss: 25802.9004
Epoch 1352/5000
1/1 [==============================] - 0s 11ms/step - loss: 3108.2810 - val_loss: 24889.1836
Epoch 1353/5000
1/1 [==============================] - 0s 11ms/step - loss: 6492.7505 - val_loss: 20644.7637
Epoch 1354/5000
1/1 [==============================] - 0s 11ms/step - loss: 11142.1064 - val_loss: 21453.2383
Epoch 1355/5000
1/1 [==============================] - 0s 11ms/step - loss: 8447.1465 - val_loss: 26204.7812
Epoch 1356/5000
1/1 [========

Epoch 1422/5000
1/1 [==============================] - 0s 12ms/step - loss: 10279.7803 - val_loss: 25361.2617
Epoch 1423/5000
1/1 [==============================] - 0s 11ms/step - loss: 18297.4668 - val_loss: 34152.1406
Epoch 1424/5000
1/1 [==============================] - 0s 12ms/step - loss: 11293.5420 - val_loss: 32804.4102
Epoch 1425/5000
1/1 [==============================] - 0s 12ms/step - loss: 16312.3545 - val_loss: 41455.6836
Epoch 1426/5000
1/1 [==============================] - 0s 12ms/step - loss: 13057.7842 - val_loss: 47254.8633
Epoch 1427/5000
1/1 [==============================] - 0s 12ms/step - loss: 13176.3818 - val_loss: 40833.4453
Epoch 1428/5000
1/1 [==============================] - 0s 12ms/step - loss: 11913.3291 - val_loss: 37259.6484
Epoch 1429/5000
1/1 [==============================] - 0s 12ms/step - loss: 11367.3447 - val_loss: 32973.2734
Epoch 1430/5000
1/1 [==============================] - 0s 12ms/step - loss: 20421.7227 - val_loss: 35164.5820
Epoch 1431

1/1 [==============================] - 0s 12ms/step - loss: 10016.6611 - val_loss: 26055.4062
Epoch 1497/5000
1/1 [==============================] - 0s 12ms/step - loss: 10724.6621 - val_loss: 26917.6250
Epoch 1498/5000
1/1 [==============================] - 0s 12ms/step - loss: 10638.0596 - val_loss: 27469.6230
Epoch 1499/5000
1/1 [==============================] - 0s 11ms/step - loss: 6569.6040 - val_loss: 23769.6445
Epoch 1500/5000
1/1 [==============================] - 0s 12ms/step - loss: 22733.7188 - val_loss: 29380.8984
Epoch 1501/5000
1/1 [==============================] - 0s 12ms/step - loss: 10478.9697 - val_loss: 28984.4805
Epoch 1502/5000
1/1 [==============================] - 0s 12ms/step - loss: 10532.5820 - val_loss: 28600.4160
Epoch 1503/5000
1/1 [==============================] - 0s 12ms/step - loss: 19275.9785 - val_loss: 38724.4453
Epoch 1504/5000
1/1 [==============================] - 0s 12ms/step - loss: 11192.3545 - val_loss: 34295.7227
Epoch 1505/5000
1/1 [======

In [45]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 44ms/step
[2022.411](test_target) - [[1853.5889]](prediction) = 168.82213281250006


In [46]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [47]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Pernambuco - IDH Longevidade,Pernambuco - Desemprego,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Pernambuco - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pernambuco - IDH Educacao,Pernambuco - IDH
0,-1.413970,-1.353481,-1.263835,1.307925,1.373937,1.383768,-0.302344,1.165029,1.396038,1.335266,...,1.190033,1.220504,0.375388,-1.321715,-1.408961,-1.223866,-1.059683,-1.391503,-1.406497,-1.414146
1,0.684239,0.321681,0.082336,-0.188132,-0.396757,-0.439118,1.347601,0.111769,-0.502296,-0.264151,...,0.066698,0.008437,0.993116,0.225194,0.809940,-0.001755,-0.281210,0.477141,0.575480,0.695116
2,0.729731,1.031800,1.181500,-1.119794,-0.977180,-0.944650,-1.045256,-1.276798,-0.893743,-1.071116,...,-1.256731,-1.228942,-1.368504,1.096521,0.599021,1.225622,1.340893,0.914362,0.831017,0.719030


train_target:


,Pernambuco - Consumo de Cimento (t)
0,2632.837886
1,2308.455257
2,1813.941000


test_input:


...

,Pernambuco - IDH Longevidade,Pernambuco - Desemprego,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Pernambuco - Produção de Cimento (t),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pernambuco - IDH Educacao,Pernambuco - IDH
6,-0.302648,-0.418301,-2.061765,0.476989,-1.758782,-1.868052,-1.621024,1.16956,2.035192,0.985175,...,-0.906462,-0.636957,-1.780396,-1.468752,-0.697983,1.593244,1.949457,0.935438,0.346973,-0.170454


test_target:


,Pernambuco - Consumo de Cimento (t)
6,2022.411


Epoch 1509: early stopping
1/1 [==============================] - 0s 30ms/step
Error: 168.82213281250006




[2055.34375, 1712.0389404296875, 1749.1181640625, 1853.5888671875]

In [48]:
display(targets)
display(predictions)

[1645.004, 1690.846, 2018.834, 2022.411]

[2055.34375, 1712.0389404296875, 1749.1181640625, 1853.5888671875]

In [49]:
mae = mean_absolute_error(predictions, targets)
mae

217.51766479492193

In [50]:
porcentage = mae/np.mean(targets)
porcentage

0.11794217902571238